<a href="https://colab.research.google.com/github/UjalaJha/nova/blob/develop/notebook/content_based_filtering_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#import libraries
import pandas as pd
import numpy as np
import random as rd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from google.colab import drive
from sklearn.preprocessing import LabelEncoder


In [4]:
# Mount Google Drive folder
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:

# Path to file in mounted Google Drive folder
file_path = '/content/drive/MyDrive/Grace Hopper/combined-1kAbove-stars.csv'

# Read CSV file using pandas
data = pd.read_csv(file_path)
data.head()

,Unnamed: 0.1,Unnamed: 0,id,node_id,name,full_name,private,owner,html_url,description,...,visibility,forks,open_issues,watchers,default_branch,permissions,score,lang_name,lang_perc,license_name
0,0,0,45717250,MDEwOlJlcG9zaXRvcnk0NTcxNzI1MA==,tensorflow,tensorflow/tensorflow,False,"{'login': 'tensorflow', 'id': 15658638, 'node_...",https://github.com/tensorflow/tensorflow,An Open Source Machine Learning Framework for ...,...,public,88115,2216,173348,master,"{'admin': False, 'maintain': False, 'push': Fa...",1.0,"['C++', 'Python', 'MLIR', 'Starlark', 'HTML', ...","[132095537, 43756548, 13084966, 8623743, 46864...",Apache-2.0
1,1,1,20580498,MDEwOlJlcG9zaXRvcnkyMDU4MDQ5OA==,kubernetes,kubernetes/kubernetes,False,"{'login': 'kubernetes', 'id': 13629408, 'node_...",https://github.com/kubernetes/kubernetes,Production-Grade Container Scheduling and Mana...,...,public,35807,2486,97468,master,"{'admin': False, 'maintain': False, 'push': Fa...",1.0,"['Go', 'Shell', 'PowerShell', 'Makefile', 'Doc...","[66222388, 1836554, 144474, 63695, 49417, 2373...",Apache-2.0
2,2,2,155220641,MDEwOlJlcG9zaXRvcnkxNTUyMjA2NDE=,transformers,huggingface/transformers,False,"{'login': 'huggingface', 'id': 25720743, 'node...",https://github.com/huggingface/transformers,🤗 Transformers: State-of-the-art Machine Learn...,...,public,19596,672,93079,main,"{'admin': False, 'maintain': False, 'push': Fa...",1.0,"['Python', 'Cuda', 'Shell', 'Dockerfile', 'C++...","[42853265, 175419, 30374, 20516, 12959, 6021, ...",Apache-2.0
3,3,3,20929025,MDEwOlJlcG9zaXRvcnkyMDkyOTAyNQ==,TypeScript,microsoft/TypeScript,False,"{'login': 'microsoft', 'id': 6154722, 'node_id...",https://github.com/microsoft/TypeScript,TypeScript is a superset of JavaScript that co...,...,public,11681,6108,90382,main,"{'admin': False, 'maintain': False, 'push': Fa...",1.0,"['TypeScript', 'JavaScript', 'Dockerfile', 'Sh...","[87866426, 198563, 330, 53]",Apache-2.0
4,4,4,90796663,MDEwOlJlcG9zaXRvcnk5MDc5NjY2Mw==,puppeteer,puppeteer/puppeteer,False,"{'login': 'puppeteer', 'id': 6906516, 'node_id...",https://github.com/puppeteer/puppeteer,Headless Chrome Node.js API,...,public,8877,268,82712,main,"{'admin': False, 'maintain': False, 'push': Fa...",1.0,"['TypeScript', 'JavaScript', 'HTML', 'CSS', 'D...","[1859434, 112872, 25585, 6581, 1574, 600]",Apache-2.0


In [21]:
data.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'id', 'node_id', 'name', 'full_name',
       'private', 'owner', 'html_url', 'description', 'fork', 'url',
       'forks_url', 'keys_url', 'collaborators_url', 'teams_url', 'hooks_url',
       'issue_events_url', 'events_url', 'assignees_url', 'branches_url',
       'tags_url', 'blobs_url', 'git_tags_url', 'git_refs_url', 'trees_url',
       'statuses_url', 'languages_url', 'stargazers_url', 'contributors_url',
       'subscribers_url', 'subscription_url', 'commits_url', 'git_commits_url',
       'comments_url', 'issue_comment_url', 'contents_url', 'compare_url',
       'merges_url', 'archive_url', 'downloads_url', 'issues_url', 'pulls_url',
       'milestones_url', 'notifications_url', 'labels_url', 'releases_url',
       'deployments_url', 'created_at', 'updated_at', 'pushed_at', 'git_url',
       'ssh_url', 'clone_url', 'svn_url', 'homepage', 'size',
       'stargazers_count', 'watchers_count', 'language', 'has_issues',
       'has_projects', 'ha

# Preprocess Data

In [22]:
# Data Picking and Analysis
from IPython.display import display
data_selected = data[["full_name","name","description","stargazers_count","forks_count","topics","language","lang_name","license_name"]]
print('\n\nAll Data -------------------------------------------------------------\n\n')
display(data_selected)
print('\n\nAll Data HEAD -------------------------------------------------------------\n\n')
display(data_selected.head())
print('\n\nAll Data Describe -------------------------------------------------------------\n\n')
display(data_selected.describe())
print('\n\nAll Data COUNT -------------------------------------------------------------\n\n')
display(data_selected.count())



All Data -------------------------------------------------------------




,full_name,name,description,stargazers_count,forks_count,topics,language,lang_name,license_name
0,tensorflow/tensorflow,tensorflow,An Open Source Machine Learning Framework for ...,173348,88115,"['deep-learning', 'deep-neural-networks', 'dis...",C++,"['C++', 'Python', 'MLIR', 'Starlark', 'HTML', ...",Apache-2.0
1,kubernetes/kubernetes,kubernetes,Production-Grade Container Scheduling and Mana...,97468,35807,"['cncf', 'containers', 'go', 'kubernetes']",Go,"['Go', 'Shell', 'PowerShell', 'Makefile', 'Doc...",Apache-2.0
2,huggingface/transformers,transformers,🤗 Transformers: State-of-the-art Machine Learn...,93079,19596,"['bert', 'deep-learning', 'flax', 'hacktoberfe...",Python,"['Python', 'Cuda', 'Shell', 'Dockerfile', 'C++...",Apache-2.0
3,microsoft/TypeScript,TypeScript,TypeScript is a superset of JavaScript that co...,90382,11681,"['javascript', 'language', 'typechecker', 'typ...",TypeScript,"['TypeScript', 'JavaScript', 'Dockerfile', 'Sh...",Apache-2.0
4,puppeteer/puppeteer,puppeteer,Headless Chrome Node.js API,82712,8877,"['automation', 'chrome', 'chromium', 'develope...",TypeScript,"['TypeScript', 'JavaScript', 'HTML', 'CSS', 'D...",Apache-2.0
...,...,...,...,...,...,...,...,...,...
5320,hashicorp/setup-terraform,setup-terraform,Sets up Terraform CLI in your GitHub Actions w...,1028,201,[],JavaScript,"['JavaScript', 'Shell']",MPL-2.0
5321,luguanxing/Cheating-Plugin-Program,Cheating-Plugin-Program,从零开始研究外挂设计原理,1025,220,[],C++,"['C++', 'C', 'Makefile']",MPL-2.0
5322,actiontech/sqle,sqle,SQLE is a SQL audit platform | SQLE 是一个支持多场景，原...,1009,125,"['advisor', 'audit', 'jetbrains', 'mssql', 'my...",Go,"['Go', 'Makefile', 'Shell', 'Dockerfile', 'Pyt...",MPL-2.0
5323,gzuidhof/starboard-notebook,starboard-notebook,In-browser literate notebooks,1008,82,"['literate-programming', 'notebook', 'starboar...",TypeScript,"['TypeScript', 'SCSS', 'JavaScript', 'CSS', 'M...",MPL-2.0




All Data HEAD -------------------------------------------------------------




,full_name,name,description,stargazers_count,forks_count,topics,language,lang_name,license_name
0,tensorflow/tensorflow,tensorflow,An Open Source Machine Learning Framework for ...,173348,88115,"['deep-learning', 'deep-neural-networks', 'dis...",C++,"['C++', 'Python', 'MLIR', 'Starlark', 'HTML', ...",Apache-2.0
1,kubernetes/kubernetes,kubernetes,Production-Grade Container Scheduling and Mana...,97468,35807,"['cncf', 'containers', 'go', 'kubernetes']",Go,"['Go', 'Shell', 'PowerShell', 'Makefile', 'Doc...",Apache-2.0
2,huggingface/transformers,transformers,🤗 Transformers: State-of-the-art Machine Learn...,93079,19596,"['bert', 'deep-learning', 'flax', 'hacktoberfe...",Python,"['Python', 'Cuda', 'Shell', 'Dockerfile', 'C++...",Apache-2.0
3,microsoft/TypeScript,TypeScript,TypeScript is a superset of JavaScript that co...,90382,11681,"['javascript', 'language', 'typechecker', 'typ...",TypeScript,"['TypeScript', 'JavaScript', 'Dockerfile', 'Sh...",Apache-2.0
4,puppeteer/puppeteer,puppeteer,Headless Chrome Node.js API,82712,8877,"['automation', 'chrome', 'chromium', 'develope...",TypeScript,"['TypeScript', 'JavaScript', 'HTML', 'CSS', 'D...",Apache-2.0




All Data Describe -------------------------------------------------------------




,stargazers_count,forks_count
count,5325.000000,5325.000000
mean,10337.564883,1828.214836
std,15594.820151,3977.591607
min,1001.000000,0.000000
25%,2293.000000,305.000000
50%,5931.000000,742.000000
75%,12821.000000,1879.000000
max,364450.000000,88115.000000




All Data COUNT -------------------------------------------------------------




full_name           5325
name                5324
description         5280
stargazers_count    5325
forks_count         5325
topics              5325
language            4945
lang_name           5325
license_name        5325
dtype: int64

In [23]:
print('\n\nUnique Language -------------------------------------------------------------\n\n')
display(data_selected["language"].unique())
print('\n\nUnique License -------------------------------------------------------------\n\n')
display(data_selected["license_name"].unique())
print('\n\nNA Language -------------------------------------------------------------\n\n')
display(data_selected["language"].isna().sum())
print('\n\nNA License -------------------------------------------------------------\n\n')
display(data_selected["license_name"].isna().sum())
# Count NaN values in multiple columns of DataFrame

print('\n\nNA Values by Column -------------------------------------------------------------\n\n')
display(data_selected.isna().sum())
display(data_selected.isna().sum().sum())



Unique Language -------------------------------------------------------------




array(['C++', 'Go', 'Python', 'TypeScript', 'C', 'Java', 'Rust',
       'Dockerfile', nan, 'Kotlin', 'JavaScript', 'Shell', 'Scala',
       'SCSS', 'Lua', 'HTML', 'Jupyter Notebook', 'Swift', 'Elixir',
       'Vue', 'Crystal', 'Makefile', 'Objective-C', 'Mustache', 'VimL',
       'WebAssembly', 'Starlark', 'Groovy', 'Markdown', 'Dart', 'Jinja',
       'PHP', 'Pascal', 'Ruby', 'Erlang', 'C#', 'PowerShell', 'Cython',
       'CSS', 'Assembly', 'Svelte', 'Prolog', 'Scheme', 'Jsonnet',
       'PLpgSQL', 'Handlebars', 'Vim Script', 'Emacs Lisp', 'Perl',
       'Matlab', 'Nim', 'TeX', 'ASL', 'CMake', 'Factor', 'Haskell',
       'YARA', 'MATLAB', 'CoffeeScript', 'Elm', 'Dhall', 'Odin',
       'AppleScript', 'Batchfile', 'Vim script', 'PostScript', 'Pawn',
       'Verilog', 'Cuda', 'Solidity', 'SVG', 'DIGITAL Command Language',
       'JSON', 'R', 'Clojure', 'Objective-C++', 'Vala', 'D', 'XSLT',
       'Mercury', 'HLSL', 'Smali', 'Visual Basic .NET', 'AutoHotkey',
       'Blade', 'OCaml', 'Haxe



Unique License -------------------------------------------------------------




array(['Apache-2.0', 'BSD-2-Clause', 'BSD-3-Clause', 'CC0-1.0', 'EPL-2.0',
       'GPL-3.0', 'LGPL-3.0', 'MIT', 'MPL-2.0'], dtype=object)



NA Language -------------------------------------------------------------




...



NA License -------------------------------------------------------------




0



NA Values by Column -------------------------------------------------------------




full_name             0
name                  1
description          45
stargazers_count      0
forks_count           0
topics                0
language            380
lang_name             0
license_name          0
dtype: int64

426

In [24]:
#This is funny the Repo is named NULL and hence its name is detected in NA LOL
df_name_na=data_selected[data_selected["name"].isna()]
display(df_name_na)

,full_name,name,description,stargazers_count,forks_count,topics,language,lang_name,license_name
1270,guregu/null,NaN,reasonable handling of nullable values,1546,224,"['go', 'json', 'sql']",Go,['Go'],BSD-2-Clause


In [25]:
df_na = data_selected[data_selected.isna().any(axis=1)]
display(df_na)


,full_name,name,description,stargazers_count,forks_count,topics,language,lang_name,license_name
24,google/material-design-icons,material-design-icons,Material Design icons by Google,47889,9587,"['android', 'icons', 'ios', 'material', 'mater...",NaN,[],Apache-2.0
45,kelseyhightower/kubernetes-the-hard-way,kubernetes-the-hard-way,Bootstrap Kubernetes the hard way on Google Cl...,34994,12019,[],NaN,[],Apache-2.0
62,Trinea/android-open-project,android-open-project,A categorized collection of Android Open Sourc...,31202,12011,"['android', 'open-source-project']",NaN,[],Apache-2.0
105,veggiemonk/awesome-docker,awesome-docker,:whale: A curated list of Docker resources and...,25006,2742,"['awesome', 'awesome-list', 'container', 'dock...",NaN,[],Apache-2.0
201,ZhongFuCheng3y/athena,athena,Java后端知识图谱🔥 帮助Java初学者成长,17833,3799,[],NaN,[],Apache-2.0
...,...,...,...,...,...,...,...,...,...
5105,mozilla/geckodriver,geckodriver,WebDriver for Firefox,6502,1522,"['firefox', 'gecko', 'geckodriver', 'rust', 'w...",NaN,[],MPL-2.0
5175,PizzaDeDados/datascience-pizza,datascience-pizza,🍕 Repositório para juntar informações sobre ma...,2233,479,"['dados', 'data-science', 'data-scientists', '...",NaN,[],MPL-2.0
5202,yadox666/The-Hackers-Hardware-Toolkit,The-Hackers-Hardware-Toolkit,The best hacker's gadgets for Red Team pentest...,1962,273,[],NaN,[],MPL-2.0
5248,hashicorp/best-practices,best-practices,NaN,1494,433,[],HCL,"['HCL', 'Shell', 'PowerShell', 'JavaScript']",MPL-2.0


In [51]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [85]:
repo_data = data_selected.copy()
display(repo_data.head())
display(type(repo_data['topics']))

,full_name,name,description,stargazers_count,forks_count,topics,language,lang_name,license_name
0,tensorflow/tensorflow,tensorflow,An Open Source Machine Learning Framework for ...,173348,88115,"['deep-learning', 'deep-neural-networks', 'dis...",C++,"['C++', 'Python', 'MLIR', 'Starlark', 'HTML', ...",Apache-2.0
1,kubernetes/kubernetes,kubernetes,Production-Grade Container Scheduling and Mana...,97468,35807,"['cncf', 'containers', 'go', 'kubernetes']",Go,"['Go', 'Shell', 'PowerShell', 'Makefile', 'Doc...",Apache-2.0
2,huggingface/transformers,transformers,🤗 Transformers: State-of-the-art Machine Learn...,93079,19596,"['bert', 'deep-learning', 'flax', 'hacktoberfe...",Python,"['Python', 'Cuda', 'Shell', 'Dockerfile', 'C++...",Apache-2.0
3,microsoft/TypeScript,TypeScript,TypeScript is a superset of JavaScript that co...,90382,11681,"['javascript', 'language', 'typechecker', 'typ...",TypeScript,"['TypeScript', 'JavaScript', 'Dockerfile', 'Sh...",Apache-2.0
4,puppeteer/puppeteer,puppeteer,Headless Chrome Node.js API,82712,8877,"['automation', 'chrome', 'chromium', 'develope...",TypeScript,"['TypeScript', 'JavaScript', 'HTML', 'CSS', 'D...",Apache-2.0


pandas.core.series.Series

In [71]:
import ast
for topic in repo_data['topics'].head():
    display(type(topic))
    topic = ast.literal_eval(topic)
    display(topic[0])
    display(type(topic))

str

'deep-learning'

list

str

'cncf'

...

'javascript'

list

str

'automation'

list

In [ ]:
repo_data['topics'] = repo_data['topics'].apply(lambda x: ast.literal_eval(x))
repo_data['topics'] = repo_data['topics'].apply(lambda x: ' '.join(x))
repo_data['topics'] = repo_data['topics'].apply(lambda x: x.lower())

repo_data['lang_name'] = repo_data['lang_name'].apply(lambda x: ast.literal_eval(x))
repo_data['lang_name'] = repo_data['lang_name'].apply(lambda x: ' '.join(x))
repo_data['lang_name'] = repo_data['lang_name'].apply(lambda x: x.lower())


In [92]:
repo_data['language'] = repo_data['language'].astype(str).apply(lambda x: x.lower())


In [93]:
display(repo_data.head())

,full_name,name,description,stargazers_count,forks_count,topics,language,lang_name,license_name
0,tensorflow/tensorflow,tensorflow,An Open Source Machine Learning Framework for ...,173348,88115,deep-learning deep-neural-networks distributed...,c++,c++ python mlir starlark html go c java jupyte...,Apache-2.0
1,kubernetes/kubernetes,kubernetes,Production-Grade Container Scheduling and Mana...,97468,35807,cncf containers go kubernetes,go,go shell powershell makefile dockerfile python...,Apache-2.0
2,huggingface/transformers,transformers,🤗 Transformers: State-of-the-art Machine Learn...,93079,19596,bert deep-learning flax hacktoberfest jax lang...,python,python cuda shell dockerfile c++ c cython make...,Apache-2.0
3,microsoft/TypeScript,TypeScript,TypeScript is a superset of JavaScript that co...,90382,11681,javascript language typechecker typescript,typescript,typescript javascript dockerfile shell,Apache-2.0
4,puppeteer/puppeteer,puppeteer,Headless Chrome Node.js API,82712,8877,automation chrome chromium developer-tools hea...,typescript,typescript javascript html css dockerfile shell,Apache-2.0


# Extract features

In [95]:
vectorizer = CountVectorizer()
topics_matrix = vectorizer.fit_transform(repo_data['topics'])
languages_matrix = vectorizer.fit_transform(repo_data['lang_name'])


# Compute similarity

In [97]:
topics_similarity = cosine_similarity(topics_matrix)
languages_similarity = cosine_similarity(languages_matrix)

In [100]:
display(topics_similarity)
display(len(topics_similarity))

array([[1.        , 0.        , 0.30117352, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.30117352, 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

5325

In [101]:
display(languages_similarity)
display(len(languages_similarity))

array([[1.        , 0.44095855, 0.35714286, ..., 0.42257713, 0.        ,
        0.21821789],
       [0.44095855, 1.        , 0.50395263, ..., 0.74535599, 0.        ,
        0.38490018],
       [0.35714286, 0.50395263, 1.        , ..., 0.6761234 , 0.        ,
        0.43643578],
       ...,
       [0.42257713, 0.74535599, 0.6761234 , ..., 1.        , 0.        ,
        0.51639778],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.25819889],
       [0.21821789, 0.38490018, 0.43643578, ..., 0.51639778, 0.25819889,
        1.        ]])

5325

# Recommendation Engine

In [110]:
def recommend_repo(topics, languages, n=10):
    # Find repos with similar topics and languages
    topics_list = topics.split(',')
    topics_indices = repo_data[repo_data['topics'].str.contains(topics)].index
    languages_indices = repo_data[repo_data['lang_name'].str.contains(languages)].index
    similar_repo = set(topics_indices) & set(languages_indices)
    
    # Compute repo similarity scores
    similarity_scores = []
    for repo in similar_repo:
        topics_similarities = topics_similarity[repo, list(similar_repo)]
        languages_similarities = languages_similarity[repo, list(similar_repo)]
        similarity_scores.append((repo, topics_similarities.mean(), languages_similarities.mean()))
    
    # Sort by similarity scores and recommend top N movies
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1] + x[2], reverse=True)[:n]
    recommended_repos = [repo_data.iloc[score[0]]['full_name'] for score in similarity_scores]
    
    return recommended_repos

In [111]:
recommend_repo('deep-learning','typescript',n=5)

['Lightning-AI/lightning',
 'tensorflow/tfjs-core',
 'streamlit/streamlit',
 'tensorflow/tfjs',
 'ray-project/ray']

In [112]:
recommend_repo('deep-learning','python',n=5)

['tensorpack/tensorpack',
 'instillai/TensorFlow-Course',
 'google/trax',
 'keras-team/keras',
 'eriklindernoren/ML-From-Scratch']

In [113]:
recommend_repo('kubernetes','go',n=5)

['derailed/k9s',
 'kubernetes/kompose',
 'kubernetes/community',
 'jtblin/kube2iam',
 'helm/charts']

In [115]:
recommend_repo('spring','java',n=5)

['codecentric/spring-boot-admin',
 'wuyouzhuguli/SpringAll',
 'macrozheng/mall',
 'apolloconfig/apollo',
 'alibaba/spring-cloud-alibaba']